In [5]:
# --- Final Optimized GA (with direct A,B,C output) ---

from google.colab import files
import pandas as pd
import numpy as np
import random
from dataclasses import dataclass

# --- Upload dataset ---
uploaded = files.upload()  # choose hybrid_composites_data.csv
filename = list(uploaded.keys())[0]
df = pd.read_csv(filename)

# --- Models ---
def flexural_strength(x):
    A, B, C = x[:,0], x[:,1], x[:,2]
    return (74.7266 - 11.4637*A + 1.9746*B - 0.3258*C +
            1.1282*A**2 - 0.0566*B**2 + 0.0014*C**2 -
            0.1125*A*B - 0.0022*A*C + 0.0015*B*C)

def impact_strength(x):
    A, B, C = x[:,0], x[:,1], x[:,2]
    return (12.1564 + 5.7193*A + 1.9796*B + 0.0062*C -
            0.1261*A**2 - 0.0342*B**2 + 0.0004*C**2 -
            0.0576*A*B - 0.0263*A*C - 0.0061*B*C)

# --- Factor Ranges ---
factors = ["A","B","C"]
ranges = {}
for col in factors:
    mn, mx = df[col].min(), df[col].max()
    mid = (mn+mx)/2
    ranges[col] = {
        "low": (mn, mid),
        "medium": (mn + (mx-mn)/4, mx - (mx-mn)/4),
        "high": (mid, mx)
    }

# --- GA Helpers ---
def genome_to_values(genome, rng):
    mapping = ["low", "medium", "high"]
    vals = []
    for gi, col in zip(genome, factors):
        low, high = ranges[col][mapping[gi]]
        vals.append(rng.uniform(low, high))
    return np.array(vals, dtype=float)

def evaluate_population(pop, rng):
    X = np.array([genome_to_values(g, rng) for g in pop])
    return flexural_strength(X), impact_strength(X)

def normalize(values, vmin, vmax):
    return (values - vmin)/(vmax-vmin) if vmax>vmin else np.zeros_like(values)

@dataclass
class GAParams:
    pop_size:int=30
    generations:int=150
    crossover_rate:float=0.9
    mutation_rate:float=0.12
    tournament_k:int=3
    elite:int=2
    weight_F:float=0.5
    weight_I:float=0.5
    seed:int=42

# --- GA Core ---
def run_ga(params:GAParams):
    rng = random.Random(params.seed)
    pop=[[rng.randrange(3) for _ in range(3)] for _ in range(params.pop_size)]

    # normalization reference
    all_genomes=[[i,j,k] for i in range(3) for j in range(3) for k in range(3)]
    X_all=np.array([genome_to_values(g, rng) for g in all_genomes])
    F_all, I_all=flexural_strength(X_all), impact_strength(X_all)
    F_min,F_max=F_all.min(),F_all.max()
    I_min,I_max=I_all.min(),I_all.max()

    best_solution=None
    best_fitness=-1
    best_generation=-1

    for gen in range(params.generations):
        F,I=evaluate_population(pop, rng)
        F_norm,I_norm=normalize(F,F_min,F_max),normalize(I,I_min,I_max)
        fit=params.weight_F*F_norm+params.weight_I*I_norm

        # record best
        idx = np.argmax(fit)
        if fit[idx] > best_fitness:
            best_fitness = fit[idx]
            best_solution = (F[idx], I[idx], pop[idx].copy(), genome_to_values(pop[idx], rng))
            best_generation = gen+1

        # selection
        def select():
            c=rng.sample(range(len(pop)),params.tournament_k)
            return pop[max(c,key=lambda i:fit[i])].copy()
        elites=[pop[i].copy() for i in np.argsort(fit)[-params.elite:]]
        new_pop=elites
        while len(new_pop)<params.pop_size:
            p1,p2=select(),select()
            if rng.random()<params.crossover_rate:
                cp=rng.choice([1,2])
                c1,c2=p1[:cp]+p2[cp:],p2[:cp]+p1[cp:]
            else:
                c1,c2=p1.copy(),p2.copy()
            for child in (c1,c2):
                for gi in range(3):
                    if rng.random()<params.mutation_rate:
                        child[gi]=rng.randrange(3)
                if len(new_pop)<params.pop_size: new_pop.append(child)
        pop=new_pop

    return best_solution, best_generation

# --- Run GA ---
params=GAParams()
(best_F, best_I, best_genome, best_values), gen = run_ga(params)

mapping=["Low","Medium","High"]
print("\n✅ Optimized Results")
print(f"Generation Found: {gen}")
print(f"A: {mapping[best_genome[0]]} → {best_values[0]:.3f}")
print(f"B: {mapping[best_genome[1]]} → {best_values[1]:.3f}")
print(f"C: {mapping[best_genome[2]]} → {best_values[2]:.3f}")
print(f"Flexural Strength (F): {best_F:.3f}")
print(f"Impact Strength (I): {best_I:.3f}")


Saving hybrid_composites_data.csv to hybrid_composites_data (4).csv

✅ Optimized Results
Generation Found: 145
A: High → 5.340
B: High → 10.738
C: Low → 81.453
Flexural Strength (F): 43.502
Impact Strength (I): 45.815
